## Vertical Edge filter on a 40x40 image

In [100]:
from pynq import Overlay
from pynq import MMIO
from pynq import Xlnk
import numpy as np
import scipy
import bitstring
from PIL import Image
from scipy.misc import imshow, imresize, imsave
from scipy.ndimage.filters import convolve,correlate
import matplotlib.pyplot as plt

### Function to convert a RGB image to grayscale

In [101]:
def rgb2grayscale(im):
        """
            Converts RGB to Grayscale.
            @params: Input RGB image.
            @return: Grayscale image(1 channel)
        """
        if  len(im.shape) > 2:
            if im.shape[2] == 3: # Convert RGB image to Grayscale
                r, g, b = im[:, :, 0], im[:, :, 1], im[:, :, 2]
                grayscale = 0.2989 * r + 0.5870 * g + 0.1140 * b
                return grayscale
        else:
            return im

### Open Input Image
The following piece of code carries out the following
- opens image
- converts it to grayscale
- print out the shape of the image

The input image is an image with horizontal and vertical lines as shown below
![Output Image](test2.jpg)

In [102]:
img = Image.open("/home/xilinx/jupyter_notebooks/test2.jpg")
img = np.array(img)
in_img = rgb2grayscale(np.array(img))
height, width = in_img.shape
print(in_img.shape)

(40, 40)


### Load FPGA bitstream
Here we load the bitstream generated from the Vivado tool as a overlay

In [103]:
overlay = Overlay("/home/xilinx/Samwise/conv_int_200_200.bit")

### Define the Sobel filter
Here we define the Sobel filter which detects vertical edges

In [104]:
kernel_dim = 3
kernel = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
kernel = kernel.T

### Declare/Initialize array for output image
Here we declare an output array of size = 1600.
This is a 1D array

In [105]:
out_img = np.zeros((height-kernel_dim//2-1)*(width-kernel_dim//2-1))
print(out_img.shape)

(1444,)


### Address and size of the input buffer

In [106]:
in_img_addr = overlay.axi_bram_ctrl_0.mmio.base_addr
print("Input buffer for the conv accelarator is present at ", hex(in_img_addr))
print("The input buffer size is",overlay.axi_bram_ctrl_0.mmio.length)

Input buffer for the conv accelarator is present at  0x40000000
The input buffer size is 8192


### Address and size of the output buffer

In [107]:
out_img_addr = overlay.axi_bram_ctrl_1.mmio.base_addr
print("Output buffer for the conv accelarator is present at ", hex(out_img_addr))
print("The output buffer size is",overlay.axi_bram_ctrl_1.mmio.length)

Output buffer for the conv accelarator is present at  0x42000000
The output buffer size is 8192


### Write kernel to Conv. core registers
Since the kernel data is stored in the registers of the core we load the kernel data here.
The kernel is stored in registers 0x40 through 0x7f

In [108]:
y,x = kernel.shape
# Address for the config register of the core
start_addr = 0x40
for j in range(y):
    for i in range(x):
        addr = (start_addr+(j*4*x)+(i*4))
        if kernel[j][i] < 0:
            val = 0xFFFFFFFF+(kernel[j][i]+1)
        else:
            val = kernel[j][i]
        print(hex(val),"@", hex(addr))
        overlay.conv_int_0.write(addr, int(val))

0x1 @ 0x40
0x0 @ 0x44
0xffffffff @ 0x48
0x2 @ 0x4c
0x0 @ 0x50
0xfffffffe @ 0x54
0x1 @ 0x58
0x0 @ 0x5c
0xffffffff @ 0x60


### Write input image to input buffer ram


In [109]:
h,w = in_img.shape
print("Input dimensions",h,w)
start_addr = 0x00
for j in range(h):
    for i in range(w):
        addr = start_addr+((i*4)+((j*4*w)))
        overlay.axi_bram_ctrl_0.mmio.write(addr, int(in_img[j][i]))

Input dimensions 40 40


### Configure the convolution core
The following steps are carried out in the code block below
- Configure the width register
- Configure the height register
- Write 5 in the 0x00 register to start conv operation
- Poll register 0x00 to check if the operation is completed

In [110]:
# Config width and height
overlay.conv_int_0.write(0x80,width)
overlay.conv_int_0.write(0x88,height)
# Write 5 to start operation
overlay.conv_int_0.write(0x00, 5)
# wait for conv operation to complete
while True:
    if overlay.conv_int_0.read(0x00) == 6:
        break

### Read Output buffer
Read the output buffer data into numpy array - out_img. The 2's complement neagtive numbers are converted accordingly

In [111]:
# read out the processed data        
start_addr = 0x00
for i in range(out_img.shape[0]):
    addr = start_addr+(i*4)
    rd_data = overlay.axi_bram_ctrl_1.mmio.read(addr)
    if rd_data>>31 == 1:
        res = -1*(0xFFFFFFFF-(rd_data-1))
    else:
        res = rd_data
    out_img[i] = res

### Reshape the 1D array to a 2D array

In [112]:
out_img = np.reshape(out_img,(h-kernel_dim//2-1,w-kernel_dim//2-1))

### Save the output image

In [113]:
_out_img = Image.fromarray(out_img)
imsave("test2_out.jpg", _out_img)

## Output Image
The output image from output buffer is below
![Output Image](test2_out.jpg)
Here we see that the horizontal lines are not part of the image